In [1]:
from nnsight import LanguageModel
from nnsight.envoy import Envoy
from nnsight.util import WrapperModule
import torch
from typing import List, Tuple, Dict, Any

from transformers.utils import fx as tfx
import torch.fx as fx
from torch.fx import replace_pattern

/share/u/caden/.conda/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
model = LanguageModel("gpt2", device_map="cuda:0", dispatch=True)

_ = model.trace("a", trace=False)
    
attention = model._model.transformer.h[3].attn
attention_envoy = model._envoy.transformer.h[3].attn

SyntaxError: invalid syntax (4126541203.py, line 3)

In [3]:
wrapper_module = WrapperModule()
wrapper_name = 'output_wrapper'

setattr(attention, wrapper_name, wrapper_module)

attention

GPT2Attention(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
  (output_wrapper): WrapperModule()
)

In [4]:
torch._dynamo.reset()
explain_output = torch._dynamo.explain(attention)(attention_envoy._fake_inputs[0][0][0])
print(explain_output)

Graph Count: 6
Graph Break Count: 5
Op Count: 13
Break Reasons:
  Break Reason 1:
    Reason: call_function UserDefinedObjectVariable(_hook) [UnspecializedNNModuleVariable(Dropout), TupleVariable(), ConstDictVariable(), TensorVariable()] {}
    User Stack:
      <FrameSummary file /share/u/caden/.local/lib/python3.11/site-packages/transformers/models/gpt2/modeling_gpt2.py, line 331 in resume_in_forward>
      <FrameSummary file /share/u/caden/.local/lib/python3.11/site-packages/transformers/models/gpt2/modeling_gpt2.py, line 212 in _attn>
      <FrameSummary file /share/u/caden/.local/lib/python3.11/site-packages/torch/nn/modules/module.py, line 1572 in _call_impl>
  Break Reason 2:
    Reason: call_function UserDefinedObjectVariable(_hook) [UnspecializedNNModuleVariable(Dropout), TupleVariable(), ConstDictVariable(), TensorVariable()] {}
    User Stack:
      <FrameSummary file /share/u/caden/.local/lib/python3.11/site-packages/transformers/models/gpt2/modeling_gpt2.py, line 212 in _a

In [17]:
attention.__dict__

{'training': False,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('bias',
               tensor([[[[ True, False, False,  ..., False, False, False],
                         [ True,  True, False,  ..., False, False, False],
                         [ True,  True,  True,  ..., False, False, False],
                         ...,
                         [ True,  True,  True,  ...,  True, False, False],
                         [ True,  True,  True,  ...,  True,  True, False],
                         [ True,  True,  True,  ...,  True,  True,  True]]]], device='cuda:0')),
              ('masked_bias', tensor(-10000., device='cuda:0'))]),
 '_non_persistent_buffers_set': {'bias', 'masked_bias'},
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks_with_kwargs': OrderedDict([(47, True)]),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': Or

In [15]:

def custom_backend(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):

    representation = str(gm)

    gm.print_readable()
    
    return gm.forward

torch._dynamo.reset()

opt_model = torch.compile(attention, backend=custom_backend, dynamic=True)
gm = opt_model(attention_envoy._fake_inputs[0][0][0])

class GraphModule(torch.nn.Module):
    def forward(self, L_x_ : torch._subclasses.fake_tensor.FakeTensor):
        l_x_ = L_x_
        
        # File: /share/u/caden/.local/lib/python3.11/site-packages/transformers/pytorch_utils.py:101, code: size_out = x.size()[:-1] + (self.nf,)
        size = l_x_.size()
        
        # File: /share/u/caden/.local/lib/python3.11/site-packages/transformers/pytorch_utils.py:102, code: x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
        l__self___bias = self.L__self___bias
        size_1 = l_x_.size(-1)
        view = l_x_.view(-1, size_1);  l_x_ = size_1 = None
        l__self___weight = self.L__self___weight
        x = torch.addmm(l__self___bias, view, l__self___weight);  l__self___bias = view = l__self___weight = None
        
        # File: /share/u/caden/.local/lib/python3.11/site-packages/transformers/pytorch_utils.py:103, code: x = x.view(size_out)
        x_1 = x.view((1, 1, 2304));  x = None
        return (x_1,)
    